#Importing the libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
!git clone https://github.com/laxmimerit/male-female-face-dataset.git

In [ ]:
epochs = 50
lr = 1e-3
batch_size = 128
data =[]
labels = []



In [ ]:
size = 224

In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.4,height_shift_range=0.4,
                                   zoom_range=0.3,
                                   rotation_range = 20,
                                   rescale = 1/255,
                                   )

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
target_size = (size,size)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory='/content/male-female-face-dataset/Training',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
validation_generator = test_gen.flow_from_directory(
    directory='/content/male-female-face-dataset/Validation',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
x,y = train_generator.next()
x.shape

## Build ML model

In [ ]:
model = Sequential()
model.add(InceptionV3(include_top=False, pooling='avg', weights ='imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
len((train_generator.filenames)), batch_size, len((train_generator.filenames))//batch_size

In [ ]:
model.fit(train_generator,steps_per_epoch=len(train_generator.filenames)//batch_size,
         epochs=1, validation_data=validation_generator,validation_steps=len(validation_generator.filenames)//batch_size) 

#Testing the Model


In [ ]:
img_path = '/content/lmao.jpg'


In [ ]:
img = load_img(img_path, target_size=(size, size, 3))
plt.imshow(img)

In [ ]:
img = img_to_array(img)

In [ ]:
img = img/255.0

In [ ]:
img = img.reshape(1, size, size, 3)

In [ ]:
model.predict(img)

In [ ]:
train_generator.class_indices

In [ ]:
def get_classes(data):
  prob = model.predict(img)[0][0]

  if prob<=0.5:
    return 'female',(1-prob)
  else:
    return 'male', prob  

# Predicted whether male or female with it's corresponding accuracy

In [ ]:
get_classes(img)

# Real-Time Prediction using Webcam

*   List item
*   List item




### Importing the libraries for using webcam

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

###Creating the working webcam functionality

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

### Saving the captured image to directory

In [ ]:
from IPython.display import Image
try:
  filename =take_photo()
  print('Saved to {}'.format(filename))
    # Show the image which was just taken.
  display (Image(filename))
except Exception as err:
# Errors will be thrown if the user does not have a webcam or if they do not
# grant the page permission to access it.
  print(str(err))

In [ ]:
img_path = '/content/photo.jpg'

In [ ]:
def get_prediction(img_path):
  img = load_img(img_path, target_size=(size, size, 3))
  plt.imshow(img)
  img = img_to_array(img)
  img = img/255.0
  img = img.reshape(1, size, size, 3)
  pred, prob = get_classes (img)
  return pred, prob

In [ ]:
get_prediction(img_path)